<a href="https://colab.research.google.com/github/hr1378/ML-Projects/blob/main/06_transfer_learning_in_tensorflow_part_3_scaling_upmine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip


--2024-11-23 07:03:25--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.128.207, 74.125.143.207, 173.194.69.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.128.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G  24.6MB/s    in 68s     

2024-11-23 07:04:33 (22.9 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [ ]:
import zipfile
ziperef=zipfile.ZipFile("/content/101_food_classes_10_percent.zip")
ziperef.extractall()
ziperef.close()

In [ ]:
train="101_food_classes_10_percent/train/"
test="101_food_classes_10_percent/test/"

In [ ]:
import tensorflow as tf
img_sz=(224,224)
train10=tf.keras.preprocessing.image_dataset_from_directory(train,
                                                            label_mode="categorical",
                                                            image_size=img_sz,
                                                            shuffle=False)

test=tf.keras.preprocessing.image_dataset_from_directory(test,
                                                           label_mode="categorical",
                                                           image_size=img_sz,
                                                           shuffle=False)

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


In [ ]:
from tensorflow.keras.layers import RandomFlip,RandomHeight,RandomWidth,RandomZoom,RandomRotation

In [ ]:
import tensorflow as tf

data_augmentation=tf.keras.Sequential([
    RandomFlip(0.2),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomWidth(0.2),
    RandomHeight(0.2)
])

In [ ]:
base=tf.keras.applications.EfficientNetB0(include_top=False)
base.trainable=False
inputs=tf.keras.layers.Input(shape=(224,224,3))
x=data_augmentation(inputs)
x = base(x, training=False)
x=layers.GlobalAveragePooling2D()(x)
po=layers.Dense(101,activation="softmax")(x)
model3=tf.keras.Model(inputs,po)


In [ ]:
model3.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_28 (InputLayer)          │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_2 (Sequential)            │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_3           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 101)                 │         129,381 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,178,952 (15.94 MB)

 Trainable params: 129,381 (505.39 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
model3.compile(loss="categorical_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])
history3=model3.fit(train10,
                     epochs=5,
                     validation_data=test,
                     validation_steps=int(0.15*len(test)))

Epoch 1/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 42s 121ms/step - accuracy: 0.0011 - loss: 6.0189 - val_accuracy: 0.3498 - val_loss: 3.4969
Epoch 2/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 39s 111ms/step - accuracy: 0.0967 - loss: 4.0334 - val_accuracy: 0.0834 - val_loss: 3.6994
Epoch 3/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 26s 111ms/step - accuracy: 0.0834 - loss: 3.8613 - val_accuracy: 0.2744 - val_loss: 2.8236
Epoch 4/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 41s 110ms/step - accuracy: 0.1245 - loss: 3.5908 - val_accuracy: 0.3456 - val_loss: 2.4894
Epoch 5/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 41s 110ms/step - accuracy: 0.1694 - loss: 3.3270 - val_accuracy: 0.4966 - val_loss: 1.9189


In [ ]:
len(train10)

237